# **Programmatically Generating Chess Training Datasets using PyQt5 and Lichess**

This notebook demonstrates how to programmatically generate training datasets for a chess neural network by capturing
chessboard screenshots from Lichess using custom FEN configurations. The screenshots are captured in different
configurations with varying themes, piece sets, and backgrounds, allowing for a diverse training dataset.

The goal is to create a dataset by generating FENs (Forsyth-Edwards Notation), capturing the corresponding chessboard
images from Lichess, and then splitting them into smaller image tiles that can be fed into a neural network for
training purposes. This is an improvement over manually capturing screenshots and ensures more comprehensive coverage
of possible chessboard configurations.

Previously, the training data consisted of manually captured screenshots of starter boards. This new approach allows
for programmatic FEN generation, automated screenshot capture, and dataset creation with better accuracy and coverage.

In [ ]:
!pip install PyQt5
!pip install PyQtWebEngine
!pip install Pillow
import sys, os
import numpy as np
from PyQt5.QtWidgets import QApplication
from PyQt5.QtCore import Qt, QUrl, QTimer
from PyQt5.QtWebEngineWidgets import QWebEngineView, QWebEngineSettings
from PyQt5.QtGui import QImage, QPainter  # Import necessary modules
from PIL import Image
import urllib.parse

# **Generating a Random FEN (Forsyth-Edwards Notation)**
This function, getRandomFEN(), generates a random FEN string representing a random arrangement of chess pieces on a chessboard. It selects from standard chess piece symbols (kings, queens, rooks, bishops, knights, pawns for both black and white) or empty squares, places them on an 8x8 grid, and formats the result as a FEN string. This can be used for creating randomized chess positions for simulations, testing, or training datasets.

In [ ]:
def getRandomFEN():
    fen_chars = list('1KQRBNPkqrbnp')
    pieces = np.random.choice(fen_chars, 64)
    fen = '/'.join([''.join(pieces[i*8:(i+1)*8]) for i in range(8)])
    return fen

# **Automated Screenshot Capture for Random Chess Board Configurations**
This Screenshot class, which extends QWebEngineView, is designed to automate the process of capturing screenshots of chessboards based on random FEN configurations. It iterates over various themes, piece sets, and backgrounds from lichess.org to generate training datasets for machine learning models.



*   The `capture()` method loads a webpage with a chessboard setup from a given URL and captures the rendered board as an image.
*   The `on_loaded()` and `take_screenshot()` methods ensure the webpage is fully rendered before taking the screenshot and saving it.
*   The `process_next()` method iterates over all the possible combinations of themes, pieces, and backgrounds, updating the FEN, and capturing new screenshots, repeating the process until all configurations are covered.

This class is useful for programmatically generating diverse chessboard images as data for training AI models in chess recognition tasks.

In [ ]:
class Screenshot(QWebEngineView):

    def __init__(self):
        super().__init__()
        self.current_index = 0
        self.total_screenshots = 0
        self.output_filenames = []
        self.theme_index = 0
        self.piece_index = 0
        self.background_index = 0
        self.themes = []
        self.pieceSets = []
        self.backgrounds = []
        self.out_folder = ''

    def capture(self, url, output_file, total_screenshots, output_filenames, themes, pieceSets, backgrounds, out_folder):
        self.output_file = output_file
        self.total_screenshots = total_screenshots
        self.output_filenames = output_filenames
        self.themes = themes
        self.pieceSets = pieceSets
        self.backgrounds = backgrounds
        self.out_folder = out_folder
        print(f"Loading URL: {url}")
        print(f"Output file: {self.output_file}")

        # Disconnect previous connections to avoid multiple triggers
        try:
            self.loadFinished.disconnect(self.on_loaded)
        except TypeError:
            pass  # No previous connections

        self.loadFinished.connect(self.on_loaded)
        self.setAttribute(Qt.WA_DontShowOnScreen)

        # Disable scrollbars
        self.page().settings().setAttribute(QWebEngineSettings.ShowScrollBars, False)
        self.page().setBackgroundColor(Qt.transparent)
        # Load the URL
        self.load(QUrl(url))
        self.resize(1920, 1080)  # Ensure the size is set before showing
        self.show()

    def on_loaded(self, ok):
        if ok:
            # Ensure that the page is fully rendered
            QTimer.singleShot(1000, self.take_screenshot)
        else:
            print("Page failed to load.")
            QTimer.singleShot(1000, self.process_next)

    def take_screenshot(self):
        try:
            # Create an image with the size of the QWebEngineView
            image = QImage(self.size(), QImage.Format_ARGB32)
            painter = QPainter(image)
            self.render(painter)  # Render the QWebEngineView content into the image
            painter.end()
            if image.isNull():
                print("Failed to capture image")
                QTimer.singleShot(1000, self.process_next)
                return
            image.save(self.output_file)
            print(f"Screenshot saved to {self.output_file}")
            QTimer.singleShot(1000, self.process_next)
        except Exception as e:
            print(f"Error taking screenshot: {e}")
            QTimer.singleShot(1000, self.process_next)

    def process_next(self):
        if os.path.exists(self.output_file):
            try:
                im = Image.open(self.output_file).crop([488, 167, 1118, 797])
                im.save(self.output_file)
            except Exception as e:
                print(f"Error processing image: {e}")
            self.current_index += 1

            # Update indices for theme, piece set, and background before generating the next screenshot
            if self.current_index >= self.total_screenshots:
                self.current_index = 0
                self.theme_index += 1  # Increment the theme first

                # After all themes are used, move to the next piece set
                if self.theme_index >= len(self.themes):
                    self.theme_index = 0  # Reset the theme index
                    self.piece_index += 1  # Increment the piece set

                    # After all piece sets are used, move to the next background
                    if self.piece_index >= len(self.pieceSets):
                        self.piece_index = 0  # Reset the piece set index
                        self.background_index += 1  # Increment the background

                        # If all backgrounds have been used, stop the app
                        if self.background_index >= len(self.backgrounds):
                            self.app.quit()  # Quit the app when all combinations are done
                            return

            print(f"Theme: {self.themes[self.theme_index]}, Piece: {self.pieceSets[self.piece_index]}, Background: {self.backgrounds[self.background_index]}")
            fen = getRandomFEN().replace('/', '-')
            next_output_file = f"{self.out_folder}/{fen}.png"
            theme = self.themes[self.theme_index]
            piece = self.pieceSets[self.piece_index]
            background = self.backgrounds[self.background_index]

            url = f"https://lichess.org/editor/{fen.replace('-', '/')}?theme={theme}&pieceSet={piece}&bg={background}"
            print(f"# {self.current_index} : {fen}")
            print(f"\tSaved screenshot to '{next_output_file}'")
            self.capture(url, next_output_file, self.total_screenshots, self.output_filenames, self.themes, self.pieceSets, self.backgrounds, self.out_folder)
            print("\t...Success")
        else:
            print(f"File not found: {self.output_file}")
            # Wait a bit before retrying
            QTimer.singleShot(1000, self.process_next)

# **Running the Automated Screenshot Generation Application**
This script initiates the PyQt application to automate the process of capturing random chessboard screenshots based on various combinations of themes, piece sets, and backgrounds.

*   It initializes the `QApplication` and creates an instance of the `Screenshot` class.
*   Several chessboard styles are defined, including themes, piece sets, and a background.
*   The script configures the number of screenshots (`N`) to be generated per combination and ensures an output directory (`train_images`) exists.
*   For each combination, the script generates a random FEN string, constructs the corresponding URL for the Lichess board editor, and triggers the screenshot capture process.
*   After generating the screenshots, the application exits once all combinations have been processed.

This setup allows for efficient and automated generation of training data for machine learning models.

In [ ]:
app = QApplication(sys.argv)
s = Screenshot()
s.app = app

# Reduced lists for testing, you can expand them later
themes = [
    "brown", "wood", "wood2", "wood3", "wood4", "maple", "maple2",
    "blue", "blue2", "blue3", "canvas", "blue-marble",
    "green", "marble", "green-plastic", "olive", "grey", "metal",
    "newspaper", "purple", "purple-diag", "pink"
]
pieceSets = [
    "cburnett", "merida", "alpha", "pirouetti", "chessnut", "chess7",
    "reillycraig", "companion", "riohacha", "kosal", "leipzig",
    "fantasy", "spatial", "celtic", "california", "caliente", "pixel",
    "maestro", "fresca", "cardinal", "gioco", "tatiana", "staunty",
    "monarchy", "governor", "dubrovny", "icpieces", "mpchess",
    "kiwen-suwi", "letter", "shapes"
]
backgrounds = ["light"]

# Number of random screenshots to generate per combination
N = 3

out_folder = 'train_images'
if not os.path.exists(out_folder):
    os.makedirs(out_folder, exist_ok=True)

fen = getRandomFEN().replace('/', '-')
s.output_filenames = [f"{out_folder}/{fen}.png" for i in range(N)]
print(f"#0 : {fen}")
print(f"\tSaved screenshot to '{s.output_filenames[0]}'")
# URL Encoding for theme, piece, and background
theme = urllib.parse.quote(themes[0])
piece = urllib.parse.quote(pieceSets[0])
bg = urllib.parse.quote(backgrounds[0])
url = f"https://lichess.org/editor/{fen.replace('-', '/')}?theme={theme}&pieceSet={piece}&bg={bg}"
s.capture(url, s.output_filenames[0], N, s.output_filenames, themes, pieceSets, backgrounds, out_folder)
print("\t...Success")

sys.exit(app.exec_())